In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
df=pd.read_csv('clean_features.csv')

In [3]:
#calulating Correlation of "inventory" with other features. 
corr_action=df.corrwith(df['action'],method='pearson').abs()
print(corr_action)

Unnamed: 0               0.034938
Unnamed: 0.1             0.034938
action                   1.000000
year                     0.018840
month                    0.043856
                           ...   
WTI | P2 | HSK (t-20)    0.045122
year (t-20)              0.024634
month (t-20)             0.016347
day (t-20)               0.087942
weekday (t-20)           0.055592
Length: 287, dtype: float64


In [4]:
relevant_features = corr_action[corr_action>0.1]

In [5]:
relevant_features 

action                1.000000
action (t-1)          0.112927
time_spread (t-3)     0.104838
time_spread (t-4)     0.125523
time_spread (t-5)     0.133369
time_spread (t-6)     0.123635
time_spread (t-7)     0.114333
time_spread (t-18)    0.116957
time_spread (t-19)    0.125845
time_spread (t-20)    0.123492
dtype: float64

In [6]:
relevant_features.head()

action               1.000000
action (t-1)         0.112927
time_spread (t-3)    0.104838
time_spread (t-4)    0.125523
time_spread (t-5)    0.133369
dtype: float64

In [7]:
selected_df = df[relevant_features.index]  # Only getting columns having corr> 0.1 wrt output

In [8]:
pearsoncorr = selected_df.corr(method='pearson').abs()


In [9]:
columns = np.full((pearsoncorr.shape[0],), True, dtype=bool)
for i in range(pearsoncorr.shape[0]):
    for j in range(i+1, pearsoncorr.shape[0]):
        if pearsoncorr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False

In [10]:
filtered_df= selected_df.columns[columns]

In [11]:
filtered_df

Index(['action', 'action (t-1)', 'time_spread (t-3)', 'time_spread (t-18)'], dtype='object')

In [12]:
selected_df.head()

,action,action (t-1),time_spread (t-3),time_spread (t-4),time_spread (t-5),time_spread (t-6),time_spread (t-7),time_spread (t-18),time_spread (t-19),time_spread (t-20)
0,0,0,0.06,0.06,0.05,0.06,0.07,-0.40,-0.90,-0.27
1,1,0,-0.08,0.06,0.06,0.05,0.06,-0.16,-0.40,-0.90
2,0,1,0.17,-0.08,0.06,0.06,0.05,-0.19,-0.16,-0.40
3,0,0,0.17,0.17,-0.08,0.06,0.06,-0.01,-0.19,-0.16
4,0,0,0.15,0.17,0.17,-0.08,0.06,0.08,-0.01,-0.19


In [13]:
df=selected_df[selected_df.columns[selected_df.columns.isin(filtered_df)]]

In [14]:
df.head()

,action,action (t-1),time_spread (t-3),time_spread (t-18)
0,0,0,0.06,-0.40
1,1,0,-0.08,-0.16
2,0,1,0.17,-0.19
3,0,0,0.17,-0.01
4,0,0,0.15,0.08


In [15]:
x_df=df.drop(columns=['action']).values
y_df=df['action'].values

In [16]:
clf_nb = GaussianNB()
clf_nb.fit(x_df, y_df)

GaussianNB()

In [17]:
#Predict the response for test dataset
y_pred_nb = clf_nb.predict(x_df)
print("Accuracy:",metrics.accuracy_score(y_df, y_pred_nb))

Accuracy: 0.5500848896434635


In [18]:
clf_knn = NearestCentroid()
clf_knn.fit(x_df,y_df)
y_pred_knn = clf_knn.predict(x_df)
print(" KNN Accuracy:",metrics.accuracy_score(y_df, y_pred_knn))

 KNN Accuracy: 0.5500848896434635


In [19]:
clf_lr = LogisticRegression(random_state=0).fit(x_df, y_df)
y_pred_lr = clf_lr.predict(x_df)
print(" Logistic Regression Accuracy:",metrics.accuracy_score(y_df, y_pred_lr))

 Logistic Regression Accuracy: 0.5619694397283531
